三、时间信息

3.1 获取每月销量
如果本月本车型没有销量，那么销量为0

3.2 已销售时间
计算本车型已经销售的时间
如果从2012年1月至今销量为0，那么本车在本月还为上市，直到有销量的月份，已销售时间设为1，并随之间增长

3.3 上月总销量与本车型销量

3.4 去年总销量与本车型销量

3.5 本月共有多少种车型在销售

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

In [2]:
train = pd.read_csv('../../raw/CarsSaleForecast/[new] yancheng_train_20171226.csv')
test = pd.read_csv('../../raw/CarsSaleForecast/yancheng_testA_20171225.csv')
trainFeature = pd.read_csv('../../raw/LiChuan/train_feature.csv')

In [3]:
trainDate = train[['sale_date', 'sale_quantity', 'class_id']]
trainDate = trainDate.groupby(['sale_date', 'class_id']).agg('sum').reset_index()
trainDate['year'] = trainDate['sale_date'].apply(lambda x: int(str(x)[0:4]))
trainDate['month'] = trainDate['sale_date'].apply(lambda x: int(str(x)[4:6]))
# trainDate = trainDate.drop(['sale_date'], axis=1)
trainDate.head()

,sale_date,class_id,sale_quantity,year,month
0,201201,125403,90,2012,1
1,201201,136916,665,2012,1
2,201201,178529,1739,2012,1
3,201201,194450,65,2012,1
4,201201,198427,47,2012,1


In [6]:
trainSaleDate = trainFeature

# 2012年1月的销量信息
trainTmp = trainDate[(trainDate['year']==2012)&(trainDate['month']==1)]
# trainTmp = trainTmp[trainTmp['month']==1]
trainTmp.loc[:, 'how_many_month_has_sell'] = 1
sale_class_count = len(trainTmp)

trainSaleDate = pd.merge(trainSaleDate, trainTmp, on='class_id', how='left')
trainSaleDate['year'].fillna(2012.0, inplace=True)
trainSaleDate['month'].fillna(1.0, inplace=True)
# trainSaleDate['how_many_month_has_sell'].fillna(0.0, inplace=True)
trainSaleDate.loc[:, 'sale_quantity_total_last_month'] = 0
trainSaleDate.loc[:, 'sale_quantity_this_last_month'] = 0
trainSaleDate.loc[:, 'this_month_sale_class_count'] = sale_class_count

In [5]:
for month in range(2, 13):
    trainSaleDateTmp = trainFeature
    trainTmp = trainDate[(trainDate['year']==2012)&(trainDate['month']==month)]
    
    sellmonth = trainSaleDate[['class_id', 'how_many_month_has_sell', 'sale_quantity']][(trainSaleDate['year']==2012)&(trainSaleDate['month']==(month-1))]
    sellmonth.dropna(inplace=True)
    sellmonth['how_many_month_has_sell'] = sellmonth['how_many_month_has_sell']+1
    sale_quantity_total_last_month = sellmonth['sale_quantity'].sum()
    sellmonth.rename(columns={'sale_quantity': 'sale_quantity_this_last_month'}, inplace=True)
    
    trainTmp = pd.merge(trainTmp, sellmonth, on='class_id', how='left')
    trainTmp['how_many_month_has_sell'].fillna(1.0, inplace=True)
    sale_class_count = len(trainTmp)
    
    trainSaleDateTmp = pd.merge(trainSaleDateTmp, trainTmp, on='class_id', how='left')
    trainSaleDateTmp['year'].fillna(2012.0, inplace=True)
    trainSaleDateTmp['month'].fillna(month, inplace=True)
    trainSaleDateTmp.loc[:, 'sale_quantity_total_last_month'] = sale_quantity_total_last_month
    trainSaleDateTmp.loc[:, 'this_month_sale_class_count'] = sale_class_count
    
    trainSaleDate = pd.concat([trainSaleDate, trainSaleDateTmp])
    
trainSaleDate.loc[:, 'sale_quantity_total_last_year'] = 0
trainSaleDate.loc[:, 'sale_quantity_this_last_year'] = 0

In [6]:
for year in range(2013, 2017):
    for month in range(1, 13):
        trainSaleDateTmp = trainFeature
        trainTmp = trainDate[(trainDate['year']==year)&(trainDate['month']==month)]
        
        if month == 1:
            sellmonth = trainSaleDate[['class_id', 'how_many_month_has_sell', 'sale_quantity']][(trainSaleDate['year']==(year-1))&(trainSaleDate['month']==12)]
        else:
            sellmonth = trainSaleDate[['class_id', 'how_many_month_has_sell', 'sale_quantity']][(trainSaleDate['year']==year)&(trainSaleDate['month']==(month-1))]
        sellmonth.dropna(inplace=True)
        sellmonth['how_many_month_has_sell'] = sellmonth['how_many_month_has_sell']+1
        sale_quantity_total_last_month = sellmonth['sale_quantity'].sum()
        sellmonth.rename(columns={'sale_quantity': 'sale_quantity_this_last_month'}, inplace=True)
        
        sellYear = trainSaleDate[['class_id', 'sale_quantity']][((trainSaleDate['year']==(year-1))&(trainSaleDate['month']>=month))|((trainSaleDate['year']==year)&(trainSaleDate['month']<month))]
        sellYear.dropna(inplace=True)
        sellYear = sellYear.groupby('class_id').agg('sum').reset_index()
        sale_quantity_total_last_year = sellYear['sale_quantity'].sum()
        sellYear.rename(columns={'sale_quantity': 'sale_quantity_this_last_year'}, inplace=True)
        
        trainTmp = pd.merge(trainTmp, sellmonth, on='class_id', how='left')
        trainTmp['how_many_month_has_sell'].fillna(1.0, inplace=True)
        sale_class_count = len(trainTmp)
        
        trainTmp = pd.merge(trainTmp, sellYear, on='class_id', how='left')
        
        trainSaleDateTmp = pd.merge(trainSaleDateTmp, trainTmp, on='class_id', how='left')
        trainSaleDateTmp['year'].fillna(year, inplace=True)
        trainSaleDateTmp['month'].fillna(month, inplace=True)
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_month'] = sale_quantity_total_last_month
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_year'] = sale_quantity_total_last_year
        trainSaleDateTmp.loc[:, 'this_month_sale_class_count'] = sale_class_count
        
        trainSaleDate = pd.concat([trainSaleDate, trainSaleDateTmp])
        
trainSaleDate.head()

,TR_0,TR_1,TR_4,TR_5,TR_5;4,TR_6,TR_7,TR_8,TR_8;7,TR_9,...,total_quality_mean,total_quality_min,type_id_1,type_id_2,type_id_3,type_id_4,wheelbase_max,wheelbase_mean,wheelbase_min,year
0,0,0,0,0,0,1,0,0,0,0,...,0.945055,0.880342,0,0,1,0,0.973262,0.973262,0.973262,2012.0
1,1,0,1,1,0,1,0,0,0,0,...,0.621083,0.555556,0,1,0,0,0.554813,0.554813,0.554813,2012.0
2,0,0,0,1,0,0,0,0,0,0,...,0.269231,0.170940,0,0,0,1,0.521390,0.354278,0.187166,2012.0
3,0,0,1,1,0,1,1,0,0,0,...,0.559829,0.461538,1,0,0,0,0.414439,0.387701,0.360963,2012.0
4,0,0,1,1,0,1,0,0,0,0,...,0.227066,0.222222,0,1,0,0,0.323529,0.322193,0.320856,2012.0


In [7]:
for year in range(2017, 2018):
    for month in range(1, 11):
        trainSaleDateTmp = trainFeature
        trainTmp = trainDate[trainDate['year']==year]
        trainTmp = trainTmp[trainTmp['month']==month]
        
        if month == 1:
            sellmonth = trainSaleDate[['class_id', 'how_many_month_has_sell', 'sale_quantity']][(trainSaleDate['year']==(year-1))&(trainSaleDate['month']==12)]
        else:
            sellmonth = trainSaleDate[['class_id', 'how_many_month_has_sell', 'sale_quantity']][(trainSaleDate['year']==year)&(trainSaleDate['month']==(month-1))]
        sellmonth.dropna(inplace=True)
        sellmonth['how_many_month_has_sell'] = sellmonth['how_many_month_has_sell']+1
        sale_quantity_total_last_month = sellmonth['sale_quantity'].sum()
        sellmonth.rename(columns={'sale_quantity': 'sale_quantity_this_last_month'}, inplace=True)
        
        sellYear = trainSaleDate[['class_id', 'sale_quantity']][((trainSaleDate['year']==(year-1))&(trainSaleDate['month']>=month))|((trainSaleDate['year']==year)&(trainSaleDate['month']<month))]
        sellYear.dropna(inplace=True)
        sellYear = sellYear.groupby('class_id').agg('sum').reset_index()
        sale_quantity_total_last_year = sellYear['sale_quantity'].sum()
        sellYear.rename(columns={'sale_quantity': 'sale_quantity_this_last_year'}, inplace=True)
        
        trainTmp = pd.merge(trainTmp, sellmonth, on='class_id', how='left')
        trainTmp['how_many_month_has_sell'].fillna(1.0, inplace=True)
        sale_class_count = len(trainTmp)
        
        trainTmp = pd.merge(trainTmp, sellYear, on='class_id', how='left')
        
        trainSaleDateTmp = pd.merge(trainSaleDateTmp, trainTmp, on='class_id', how='left')
        trainSaleDateTmp['year'].fillna(year, inplace=True)
        trainSaleDateTmp['month'].fillna(month, inplace=True)
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_month'] = sale_quantity_total_last_month
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_year'] = sale_quantity_total_last_year
        trainSaleDateTmp.loc[:, 'this_month_sale_class_count'] = sale_class_count
        
        trainSaleDate = pd.concat([trainSaleDate, trainSaleDateTmp])

In [8]:
trainSaleDate.sort_values(['year', 'month', 'class_id'])
trainSaleDate.fillna(0.0, inplace=True)
trainSaleDate.to_csv('../../raw/LiChuan/trainSaleDate.csv', index=False)

In [9]:
# trainSaleDate['year'] = trainSaleDate['year'].astype('int32')
# trainSaleDate['mouth'] = trainSaleDate['mouth'].astype('int32')
# trainSaleDate2 = trainSaleDate[(trainSaleDate.year != 2012)]

In [ ]:
# trainSaleDate2.to_csv('../../raw/LiChuan/trainSaleDate2.csv', index=False)